In [ ]:
# 必要なライブラリをインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
from sklearn.metrics import roc_auc_score, average_precision_score

# 自作モジュールをインポート
from horgues3.dataset import HorguesDataset
from horgues3.models import HorguesModel
from horgues3.losses import HorguesLoss

In [ ]:
# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# デバイス設定
device = torch.device('cuda:1')
logger.info(f"使用デバイス: {device}")

In [ ]:
# 学習パラメータ設定
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
NUM_EPOCHS = 100
PATIENCE = 10

In [ ]:
# 学習期間・検証期間設定
TRAIN_START_DATE = '20180101'
TRAIN_END_DATE = '20211231'
VAL_START_DATE = '20220101'
VAL_END_DATE = '20221231'

logger.info(f"学習期間: {TRAIN_START_DATE} - {TRAIN_END_DATE}")
logger.info(f"検証期間: {VAL_START_DATE} - {VAL_END_DATE}")

In [ ]:
# データセットの作成
logger.info("学習データセットを作成中...")
train_dataset = HorguesDataset(
    start_date=TRAIN_START_DATE,
    end_date=TRAIN_END_DATE,
    cache_dir='cache/train',
    use_cache=True
)

logger.info("検証データセットを作成中...")
val_dataset = HorguesDataset(
    start_date=VAL_START_DATE,
    end_date=VAL_END_DATE,
    preprocessing_params=train_dataset.get_preprocessing_params(),
    cache_dir='cache/val',
    use_cache=True
)

logger.info(f"学習データサイズ: {len(train_dataset)}")
logger.info(f"検証データサイズ: {len(val_dataset)}")

In [ ]:
# データローダーの作成
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=8,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
    pin_memory=True
)

In [ ]:
# モデル設定の取得
model_config = train_dataset.get_model_config()
logger.info(f"数値特徴量数: {len(model_config['numerical_features'])}")
logger.info(f"カテゴリ特徴量数: {len(model_config['categorical_features'])}")
logger.info(f"時系列データ: {model_config['sequence_names']}")

In [ ]:
# モデルの作成
model = HorguesModel(
    sequence_names=model_config['sequence_names'],
    feature_aliases=model_config['feature_aliases'],
    numerical_features=model_config['numerical_features'],
    categorical_features=model_config['categorical_features'],
).to(device)

In [ ]:
# パラメータ数の確認
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
logger.info(f"総パラメータ数: {total_params:,}")
logger.info(f"学習可能パラメータ数: {trainable_params:,}")

In [ ]:
# 損失関数の設定
criterion = HorguesLoss()

In [ ]:
# オプティマイザーの設定
optimizer = optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=1e-5,
    betas=(0.9, 0.95)
)

In [ ]:
# スケジューラーの設定
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0=10,
    T_mult=2,
    eta_min=LEARNING_RATE * 0.01
)

In [ ]:
# 学習履歴を保存するリスト
train_losses = []
val_losses = []
learning_rates = []
best_val_loss = float('inf')
patience_counter = 0

In [ ]:
# 学習ループ
logger.info("学習を開始します...")

# 履歴を保存するためのリスト（既存のものに追加）
val_metrics_history = {
    'accuracy': {'top1': [], 'top2': [], 'top3': []},
    'precision': {'top1': [], 'top2': [], 'top3': []},
    'recall': {'top1': [], 'top2': [], 'top3': []},
    'f1': {'top1': [], 'top2': [], 'top3': []},
    'auroc': {'top1': [], 'top2': [], 'top3': []},
    'average_precision': {'top1': [], 'top2': [], 'top3': []}
}

for epoch in range(NUM_EPOCHS):
    # 学習フェーズ
    model.train()
    train_loss = 0.0
    train_batches = 0

    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} [Train]')
    for batch in train_pbar:
        optimizer.zero_grad()

        # データをデバイスに移動
        x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
        x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
        sequence_data = {}
        for seq_name, seq_data in batch['sequence_data'].items():
            sequence_data[seq_name] = {
                'x_num': {k: v.to(device) for k, v in seq_data['x_num'].items()},
                'x_cat': {k: v.to(device) for k, v in seq_data['x_cat'].items()},
                'mask': seq_data['mask'].to(device)
            }
        mask = batch['mask'].to(device)
        target = batch['target'].to(device)

        # 前向き伝播
        scores = model(x_num, x_cat, sequence_data, mask)
        loss = criterion(scores, target)

        # 後向き伝播
        loss.backward()

        # 勾配クリッピング
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        train_loss += loss.item()
        train_batches += 1

        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_train_loss = train_loss / train_batches if train_batches > 0 else 0
    train_losses.append(avg_train_loss)

    # 検証フェーズ
    model.eval()
    val_loss = 0.0
    val_batches = 0
    
    # 予測と正解を蓄積するリスト
    all_preds = []
    all_targets = []
    all_scores = []  # 確率値も蓄積

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} [Val]')
        for batch in val_pbar:
            # データをデバイスに移動
            x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
            x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
            sequence_data = {}
            for seq_name, seq_data in batch['sequence_data'].items():
                sequence_data[seq_name] = {
                    'x_num': {k: v.to(device) for k, v in seq_data['x_num'].items()},
                    'x_cat': {k: v.to(device) for k, v in seq_data['x_cat'].items()},
                    'mask': seq_data['mask'].to(device)
                }
            mask = batch['mask'].to(device)
            target = batch['target'].to(device)

            scores = model(x_num, x_cat, sequence_data, mask)
            loss = criterion(scores, target)

            val_loss += loss.item()
            val_batches += 1
            
            # 予測値を0/1に変換（シグモイド関数を適用して閾値0.5で二値化）
            scores_sigmoid = torch.sigmoid(scores)
            preds = scores_sigmoid > 0.5
            
            # バッチの予測、正解、スコアを蓄積
            all_preds.append(preds.cpu())
            all_targets.append(target.cpu())
            all_scores.append(scores_sigmoid.cpu())  # 確率値を蓄積

            val_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_val_loss = val_loss / val_batches if val_batches > 0 else float('inf')
    val_losses.append(avg_val_loss)
    
    # 全体の予測と正解、スコアを結合
    all_preds = torch.cat(all_preds, dim=0)  # [total_samples, 3]
    all_targets = torch.cat(all_targets, dim=0)  # [total_samples, 3]
    all_scores = torch.cat(all_scores, dim=0)  # [total_samples, 3]
    
    # 各順位（1着以内、2着以内、3着以内）について指標を計算
    metrics = {'accuracy': {}, 'precision': {}, 'recall': {}, 'f1': {}, 'auroc': {}, 'average_precision': {}}
    
    for i, position in enumerate(['top1', 'top2', 'top3']):
        # NaNでないサンプルのマスクを作成
        valid_mask = ~torch.isnan(all_targets[:, i])
        
        # 有効なサンプルのみを抽出
        valid_preds = all_preds[valid_mask, i]
        valid_targets = all_targets[valid_mask, i]
        valid_scores = all_scores[valid_mask, i]
        
        if len(valid_preds) > 0:
            # 各指標を計算
            tp = ((valid_preds == 1) & (valid_targets == 1)).sum().float()
            tn = ((valid_preds == 0) & (valid_targets == 0)).sum().float()
            fp = ((valid_preds == 1) & (valid_targets == 0)).sum().float()
            fn = ((valid_preds == 0) & (valid_targets == 1)).sum().float()
            
            # Accuracy
            accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
            metrics['accuracy'][position] = accuracy.item()
            
            # Precision
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            metrics['precision'][position] = precision.item()
            
            # Recall
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            metrics['recall'][position] = recall.item()
            
            # F1 Score
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            metrics['f1'][position] = f1.item()
            
            # AUROC, Average Precision (正例が存在する場合のみ計算)
            if valid_targets.sum() > 0 and valid_targets.sum() < len(valid_targets):
                try:
                    auroc = roc_auc_score(valid_targets.numpy(), valid_scores.numpy())
                    metrics['auroc'][position] = auroc
                    
                    average_precision = average_precision_score(valid_targets.numpy(), valid_scores.numpy())
                    metrics['average_precision'][position] = average_precision
                except:
                    logger.warning(f"{position}の AUROC / Average Precision 計算でエラー: {e}")
                    logger.warning(f"  正例数: {positive_count}, 負例数: {negative_count}")
                    logger.warning(f"  スコア範囲: [{y_scores.min():.3f}, {y_scores.max():.3f}]")
                    metrics['auroc'][position] = 0
                    metrics['average_precision'][position] = 0
            else:
                metrics['auroc'][position] = 0
                metrics['average_precision'][position] = 0
        else:
            # 有効なサンプルがない場合
            metrics['accuracy'][position] = 0
            metrics['precision'][position] = 0
            metrics['recall'][position] = 0
            metrics['f1'][position] = 0
            metrics['auroc'][position] = 0
            metrics['average_precision'][position] = 0
    
    # 履歴に追加
    for metric_name in ['accuracy', 'precision', 'recall', 'f1', 'auroc', 'average_precision']:
        for position in ['top1', 'top2', 'top3']:
            val_metrics_history[metric_name][position].append(metrics[metric_name][position])

    # 現在の学習率を記録
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append(current_lr)

    # スケジューラの更新
    scheduler.step()

    # ログ出力（拡張版）
    logger.info(f'Epoch {epoch+1}/{NUM_EPOCHS}: '
                f'Train Loss: {avg_train_loss:.4f}, '
                f'Val Loss: {avg_val_loss:.4f}, '
                f'LR: {current_lr:.6f}')
    
    # 各順位の指標をログ出力
    for position in ['top1', 'top2', 'top3']:
        logger.info(f'  {position.upper()} - '
                   f'Acc: {metrics["accuracy"][position]:.4f}, '
                   f'Prec: {metrics["precision"][position]:.4f}, '
                   f'Rec: {metrics["recall"][position]:.4f}, '
                   f'F1: {metrics["f1"][position]:.4f}, '
                   f'AUROC: {metrics["auroc"][position]:.4f}, '
                   f'AP: {metrics["average_precision"][position]:.4f}')

    # Best modelの保存
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0

        # モデルの保存
        model_save_path = Path('outputs')
        model_save_path.mkdir(exist_ok=True)

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'val_metrics': metrics,  # 現在のエポックの指標を保存
            'model_config': model_config,
            'preprocessing_params': train_dataset.get_preprocessing_params(),
            # 履歴データも保存
            'train_losses': train_losses,
            'val_losses': val_losses,
            'val_metrics_history': val_metrics_history,
            'learning_rates': learning_rates
        }, model_save_path / 'best_model.pth')

        logger.info(f'新しいベストモデルを保存しました (Val Loss: {best_val_loss:.4f})')
    else:
        patience_counter += 1

    # Early stopping
    if patience_counter >= PATIENCE:
        logger.info(f'Early stopping triggered after {epoch+1} epochs')
        break

# 学習終了時に最終的な履歴を保存
history_save_path = Path('outputs')
history_save_path.mkdir(exist_ok=True)

torch.save({
    'train_losses': train_losses,
    'val_losses': val_losses,
    'val_metrics_history': val_metrics_history,
    'learning_rates': learning_rates,
    'final_epoch': epoch
}, history_save_path / 'training_history.pth')

logger.info("学習が終了しました")

In [ ]:
# 学習曲線の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue')
plt.plot(val_losses, label='Validation Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(learning_rates, label='Learning Rate', color='green')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedule')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('outputs/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# オプション: 学習曲線をプロットする関数の例
def plot_training_history(history_path='outputs/training_history.pth'):
    import matplotlib.pyplot as plt
    
    history = torch.load(history_path)
    
    fig, axes = plt.subplots(3, 3, figsize=(18, 15))
    
    # Loss
    axes[0, 0].plot(history['train_losses'], label='Train Loss')
    axes[0, 0].plot(history['val_losses'], label='Val Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training and Validation Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Accuracy
    axes[0, 1].plot(history['val_metrics_history']['accuracy']['top1'], label='Top 1')
    axes[0, 1].plot(history['val_metrics_history']['accuracy']['top2'], label='Top 2')
    axes[0, 1].plot(history['val_metrics_history']['accuracy']['top3'], label='Top 3')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Validation Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Precision
    axes[0, 2].plot(history['val_metrics_history']['precision']['top1'], label='Top 1')
    axes[0, 2].plot(history['val_metrics_history']['precision']['top2'], label='Top 2')
    axes[0, 2].plot(history['val_metrics_history']['precision']['top3'], label='Top 3')
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('Precision')
    axes[0, 2].set_title('Validation Precision')
    axes[0, 2].legend()
    axes[0, 2].grid(True)
    
    # Recall
    axes[1, 0].plot(history['val_metrics_history']['recall']['top1'], label='Top 1')
    axes[1, 0].plot(history['val_metrics_history']['recall']['top2'], label='Top 2')
    axes[1, 0].plot(history['val_metrics_history']['recall']['top3'], label='Top 3')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Recall')
    axes[1, 0].set_title('Validation Recall')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # F1 Score
    axes[1, 1].plot(history['val_metrics_history']['f1']['top1'], label='Top 1')
    axes[1, 1].plot(history['val_metrics_history']['f1']['top2'], label='Top 2')
    axes[1, 1].plot(history['val_metrics_history']['f1']['top3'], label='Top 3')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('F1 Score')
    axes[1, 1].set_title('Validation F1 Score')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    # AUROC
    axes[1, 2].plot(history['val_metrics_history']['auroc']['top1'], label='Top 1')
    axes[1, 2].plot(history['val_metrics_history']['auroc']['top2'], label='Top 2')
    axes[1, 2].plot(history['val_metrics_history']['auroc']['top3'], label='Top 3')
    axes[1, 2].set_xlabel('Epoch')
    axes[1, 2].set_ylabel('AUROC')
    axes[1, 2].set_title('Validation AUROC')
    axes[1, 2].legend()
    axes[1, 2].grid(True)
    
    # Average Precision
    axes[2, 0].plot(history['val_metrics_history']['average_precision']['top1'], label='Top 1')
    axes[2, 0].plot(history['val_metrics_history']['average_precision']['top2'], label='Top 2')
    axes[2, 0].plot(history['val_metrics_history']['average_precision']['top3'], label='Top 3')
    axes[2, 0].set_xlabel('Epoch')
    axes[2, 0].set_ylabel('Average Precision')
    axes[2, 0].set_title('Validation Average Precision')
    axes[2, 0].legend()
    axes[2, 0].grid(True)
    
    # Learning Rate
    axes[2, 1].plot(history['learning_rates'])
    axes[2, 1].set_xlabel('Epoch')
    axes[2, 1].set_ylabel('Learning Rate')
    axes[2, 1].set_title('Learning Rate Schedule')
    axes[2, 1].grid(True)
    
    # 最後のサブプロットは空にする
    axes[2, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig('outputs/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_history()



In [ ]:
# 最終結果の表示
logger.info(f"最良の検証損失: {best_val_loss:.4f}")
logger.info(f"学習データ最終損失: {train_losses[-1]:.4f}")
logger.info(f"検証データ最終損失: {val_losses[-1]:.4f}")

In [ ]:
# 学習済みモデルのロード（確認用）
logger.info("保存されたベストモデルをロードして確認...")
checkpoint = torch.load('outputs/best_model.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
logger.info(f"ベストモデル (Epoch {checkpoint['epoch']+1}) をロードしました")